In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
xls = pd.ExcelFile(f'./var/GEO402.Homework1.data.2020.xlsx.xlsx')
names = xls.sheet_names

In [ ]:
for i in range(11):
    tmax = pd.read_excel(xls, names[i*3]).replace([-99], np.nan)
    tmin = pd.read_excel(xls, names[i*3+1]).replace([-99], np.nan)
    prcp = pd.read_excel(xls, names[i*3+2]).replace([-99], np.nan)

    tmean = ((tmax + tmin) / 2.)
    tm = (tmean.values[:30, 1:-1] -32.) * 5. / 9. 
    tm = np.nanmean(tm, axis=0)

    pm = np.nanmean(prcp.values[:30, 1:-1] * 25.4, axis=0)

    X = np.arange(1, 13, 1)
    fig, ax = plt.subplots(1, 1, sharex=True, figsize=(8, 6))
    ax.bar(X, pm, facecolor='purple', edgecolor='k', width=.5)
    ax.set_ylim(0, 250)
    ax.set_ylabel('Precipitation (mm)', fontweight='bold', color='k', fontsize='large')
    ax.set_yticks(np.arange(0, 260, 50))
    ax.set_yticklabels(np.arange(0, 260, 50), color='k', fontsize='large')
    ax.set_xlabel('Month', fontweight='bold', color='k', fontsize='large')
    ax.set_xticks(np.arange(1, 13, 1))
    ax.set_xticklabels(np.arange(1, 13, 1), color='k', fontsize='large')
    ax.legend(['Precipitation'], frameon=False, bbox_to_anchor=(0.5, .5, .5, 0.5), fontsize='large')

    axx = ax.twinx()
    axx.plot(X, tm, '-*')
    axx.set_ylim(-15, 35)
    axx.set_ylabel('Temperature (C)', fontweight='bold', color='k', fontsize='large')
    axx.set_yticks(np.arange(-15, 40, 5))
    axx.set_yticklabels(np.arange(-15, 40, 5), color='k', fontsize='large')
    axx.legend(['Temperature'], frameon=False, bbox_to_anchor=(0.5, .45, .5, 0.5), fontsize='large')
    axx.set_title(f'Monthly Mean Precipitation and Temperature at {names[i*3][:-3]} 1981-2010', fontweight='bold', fontsize='large')

    plt.savefig(f'./plot/GEO402/{names[i*3][:-3]}.png', bbox_inches='tight')
    plt.close()

In [ ]:
climatecategory = []
prcpstd = []
Trange = []
FrostFreeMon = []
cv = []
for i in range(7, 8):
    tmax = pd.read_excel(xls, names[i*3]).replace([-99], np.nan)
    tmin = pd.read_excel(xls, names[i*3+1]).replace([-99], np.nan)
    prcp = pd.read_excel(xls, names[i*3+2]).replace([-99], np.nan)

    tmean = ((tmax + tmin) / 2.)
    tm = (tmean.values[:30, 1:-1] -32.) * 5. / 9. 
    MAT = tm.mean(axis=1).mean()
    tm = np.nanmean(tm, axis=0)
    Thot = np.max(tm)
    Tcold = np.min(tm)
    Tmon10 = np.where(tm>10, 1, 0).sum()
    Trange.append(Thot-Tcold)
    FrostFreeMon.append(np.where(tm>0, 1, 0))
    
    pm = prcp.values[:30, 1:-1] * 25.4
    MAP = pm.sum(axis=1).mean()
    pm = np.nanmean(pm, axis=0)
    prcpstd.append(np.std(pm))
    cv.append(np.std(pm)/pm.mean())
    
    Pdry = np.min(pm)
    Psdry = np.min(pm[3:9])
    Pwdry = np.min(pm[np.r_[0:3, 9:12]])
    Pswet = np.max(pm[3:9])
    Pwwet = np.max(pm[np.r_[0:3, 9:12]])
    if pm[np.r_[0:3, 9:12]].sum() / pm.sum() > .7:
        Pthreshold = 2. * MAT
    elif pm[3:9].sum() / pm.sum() > .7:
        Pthreshold = 2. * MAT + 28.
    else:
        Pthreshold = 2. * MAT + 14.
        
    o1 = FirstOrder(MAP, Pthreshold, Tcold, Thot)
    o2 = SecondOrder(o1, Pdry, MAP, Pthreshold, Psdry, Pwwet, Pwdry, Pswet, Thot)
    o3 = ThridOrder(o1, MAT, Thot, Tmon10, Tcold)
    
    climatecategory.append(o1+o2+o3)

In [ ]:
def FirstOrder(MAP, Pthreshold, Tcold, Thot):
    if MAP < 10*Pthreshold:
        return 'B'
    elif Tcold >= 18:
        return 'A'
    elif (Thot > 10) and (Tcold > 0) and (Tcold < 18):
        return 'C'
    elif (Thot > 10) and (Tcold <= 0):
        return 'D'
    elif Thot < 10:
        return 'E'

In [ ]:
def SecondOrder(o1, Pdry, MAP, Pthreshold, Psdry, Pwwet, Pwdry, Pswet, Thot):
    if o1 == 'A':
        if Pdry >= 60.:
            return 'f'
        elif Pdry >= (100. - MAP/25.):
            return 'm'
        elif Pdry < (100. - MAP/25.):
            return 'w'
    if o1 == 'B':
        if MAP < 5*Pthreshold:
            return 'W'
        elif MAP >= 5*Pthreshold:
            return 'S'
    if (o1 == 'C') or (o1 == 'D'):
        if (Psdry < 40) and (Psdry < Pwwet/3.):
            return 's'
        elif Pwdry < Pswet/10.:
            return 'w'
        else:
            return 'f'
    if o1 == 'E':
        if Thot > 0:
            return 'T'
        elif Thot <= 0:
            return 'F'

In [ ]:
def ThridOrder(o1, MAT, Thot, Tmon10, Tcold):
    if o1 == 'B':
        if MAT >= 18:
            return 'h'
        elif MAT < 18:
            return 'k'
    elif o1 == 'C':
        if Thot >= 22:
            return 'a'
        elif Tmon10 >= 4:
            return 'b'
        elif (Tmon10 >= 1) and (Tmon10 < 4):
            return 'c'
    elif o1 == 'D':
        if Thot >= 22:
            return 'a'
        elif Tmon10 >= 4:
            return 'b'
        elif Tcold < -38:
            return 'd'
        else:
            return 'c'
    else:
        return ''

In [ ]:
data = [MAT, MAP, Pthreshold, Tcold, Thot, Pdry, Psdry, Pwwet, Pwdry, Pswet, Tmon10]
name = ['MAT', 'MAP', 'Pthreshold', 'Tcold', 'Thot', 'Pdry', 'Psdry', 'Pwwet', 'Pwdry', 'Pswet', 'Tmon10']
pd.DataFrame({name[i]:[f'{data[i]:.2f}'] for i in range(11)})
# o1+o2+o3

In [ ]:
pd.DataFrame({names[i*3][:-3]:[f'{cv[i]:.2f}'] for i in range(11)})

In [ ]:
df = pd.DataFrame({names[i*3][:-3]:FrostFreeMon[i] for i in range(11)})
df.rename(index={i:(i+1) for i in range(12)})

<font size=4 color=blue> HW2

In [2]:
xls = pd.ExcelFile(f'./var/DianeWang_HW2.First.Freeze.Data.and.Summary.xlsx')
names = xls.sheet_names

In [3]:
si = 0
df = pd.read_excel(xls, names[si]).replace([-999], np.nan)
dates = df[df['YEAR']==(1981)]['Date']
dates = [dates.astype(str)[i][5:] for i in range(365)]

In [4]:
date32 = np.zeros((6, 30))
date28 = np.zeros((6, 30))
for si in range(6):
    df = pd.read_excel(xls, names[si]).replace([-999], np.nan)
    for yi in range(30):
        if (yi+1981)%4:
            FreezeDate = np.where(df[df['YEAR']==(1981+yi)]['TMIN'].astype(float) < 32.)[0]
            date32[si, yi] = min(np.where(FreezeDate > 212, FreezeDate, 999)) + 1

            FreezeDate = np.where(df[df['YEAR']==(1981+yi)]['TMIN'].astype(float) < 28.)[0]
            date28[si, yi] = min(np.where(FreezeDate > 212, FreezeDate, 999)) + 1
        else:
            FreezeDate = np.where(df[df['YEAR']==(1981+yi)]['TMIN'].astype(float) < 32.)[0]
            date32[si, yi] = min(np.where(FreezeDate > 212, FreezeDate, 999)) 

            FreezeDate = np.where(df[df['YEAR']==(1981+yi)]['TMIN'].astype(float) < 28.)[0]
            date28[si, yi] = min(np.where(FreezeDate > 212, FreezeDate, 999))

In [5]:
sfdate = []
for si in range(6):
    fdate = []
    fdate.append(dates[np.rint(np.mean(date32[si, :])).astype(int)])
    fdate.append(dates[np.rint(np.median(date32[si, :])).astype(int)])
    fdate.append(dates[np.rint(np.min(date32[si, :])).astype(int)])
    fdate.append(dates[np.rint(np.percentile(date32[si, :], 10)).astype(int)])
    fdate.append(dates[np.rint(np.percentile(date32[si, :], 30)).astype(int)])
    fdate.append(dates[np.rint(np.percentile(date32[si, :], 50)).astype(int)])
    fdate.append(dates[np.rint(np.percentile(date32[si, :], 70)).astype(int)])
    fdate.append(dates[np.rint(np.percentile(date32[si, :], 90)).astype(int)])
    fdate.append(dates[np.rint(np.max(date32[si, :])).astype(int)])
    sfdate.append(fdate)
    
    fdate28 = []
    fdate28.append(dates[np.rint(np.mean(date28[si, :])).astype(int)])
    fdate28.append(dates[np.rint(np.median(date28[si, :])).astype(int)])
    fdate28.append(dates[np.rint(np.min(date28[si, :])).astype(int)])
    fdate28.append(dates[np.rint(np.percentile(date28[si, :], 10)).astype(int)])
    fdate28.append(dates[np.rint(np.percentile(date28[si, :], 30)).astype(int)])
    fdate28.append(dates[np.rint(np.percentile(date28[si, :], 50)).astype(int)])
    fdate28.append(dates[np.rint(np.percentile(date28[si, :], 70)).astype(int)])
    fdate28.append(dates[np.rint(np.percentile(date28[si, :], 90)).astype(int)])
    fdate28.append(dates[np.rint(np.max(date28[si, :])).astype(int)])
    sfdate.append(fdate28)
        
pd.DataFrame(sfdate).T

,0,1,2,3,4,5,6,7,8,9,10,11
0,10-12,10-23,10-09,10-24,09-26,10-10,10-08,10-21,10-07,10-22,09-09,09-30
1,10-10,10-24,10-08,10-26,09-26,10-10,10-08,10-19,10-06,10-24,09-13,10-02
2,09-08,09-24,09-22,09-28,08-19,09-24,09-22,09-24,09-18,09-18,08-04,08-04
3,09-30,10-08,09-25,10-06,09-20,09-28,09-24,10-06,09-25,10-03,08-21,09-20
4,10-05,10-16,10-04,10-13,09-24,10-04,10-02,10-14,10-02,10-14,09-06,09-26
5,10-10,10-24,10-08,10-26,09-26,10-10,10-08,10-19,10-06,10-24,09-13,10-02
6,10-18,10-31,10-15,11-01,10-02,10-14,10-13,10-27,10-11,10-30,09-17,10-07
7,10-29,11-05,10-22,11-13,10-04,10-23,10-23,11-04,10-19,11-07,09-22,10-14
8,11-04,11-21,10-29,11-28,10-10,11-01,10-29,11-23,10-30,11-12,10-02,10-15


In [6]:
sfdate = []
for si in range(6):
    fdate = []
    fdate.append(np.rint(np.mean(date32[si, :])).astype(int))
    fdate.append(np.rint(np.median(date32[si, :])).astype(int))
    fdate.append(np.rint(np.min(date32[si, :])).astype(int))
    fdate.append(np.rint(np.percentile(date32[si, :], 10)).astype(int))
    fdate.append(np.rint(np.percentile(date32[si, :], 30)).astype(int))
    fdate.append(np.rint(np.percentile(date32[si, :], 50)).astype(int))
    fdate.append(np.rint(np.percentile(date32[si, :], 70)).astype(int))
    fdate.append(np.rint(np.percentile(date32[si, :], 90)).astype(int))
    fdate.append(np.rint(np.max(date32[si, :])).astype(int))
    sfdate.append(fdate)
    
#     fdate28 = []
#     fdate28.append(np.rint(np.mean(date28[si, :])).astype(int))
#     fdate28.append(np.rint(np.median(date28[si, :])).astype(int))
#     fdate28.append(np.rint(np.min(date28[si, :])).astype(int))
#     fdate28.append(np.rint(np.percentile(date28[si, :], 10)).astype(int))
#     fdate28.append(np.rint(np.percentile(date28[si, :], 30)).astype(int))
#     fdate28.append(np.rint(np.percentile(date28[si, :], 50)).astype(int))
#     fdate28.append(np.rint(np.percentile(date28[si, :], 70)).astype(int))
#     fdate28.append(np.rint(np.percentile(date28[si, :], 90)).astype(int))
#     fdate28.append(np.rint(np.max(date28[si, :])).astype(int))
#     sfdate.append(fdate28)
        
pd.DataFrame(sfdate).T

,0,1,2,3,4,5
0,284,281,268,280,279,251
1,282,280,268,280,278,255
2,250,264,230,264,260,215
3,272,267,262,266,267,232
4,277,276,266,274,274,248
5,282,280,268,280,278,255
6,290,287,274,285,283,259
7,301,294,276,295,291,264
8,307,301,282,301,302,274


In [7]:
table = pd.DataFrame(date32.std(axis=1)).T
table.rename(columns={i:names[i] for i in range(6)})
table

,0,1,2,3,4,5
0,12.512616,9.953001,9.824912,10.075327,9.461971,13.235516


In [8]:
table = pd.DataFrame(date28.std(axis=1)).T
table.rename(columns={i:names[i] for i in range(6)})
table

,0,1,2,3,4,5
0,12.918935,15.797363,10.194552,13.701419,14.312427,14.399846
